In [222]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [223]:
iris_dataset=pd.read_csv('iris.csv')
iris_dataset=iris_dataset.drop('Id',axis=1)
iris_dataset.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## DATA PROCESSING

In [224]:
label_encoder=LabelEncoder()
Labled_Species=iris_dataset['Species'].unique().tolist()
iris_dataset['Species']=label_encoder.fit_transform(iris_dataset['Species'])
iris_dataset

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## DATA EXPLORATION

In [225]:
iris_dataset.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [226]:
iris_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


In [227]:
iris_dataset.isna().sum()

SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64

## DATA PREPARTION AND TRAINING

In [228]:
features=iris_dataset
label=iris_dataset.pop('Species')

In [229]:
train_X,val_X,train_y,val_y=train_test_split(features,label,test_size=0.2,random_state=42)

In [230]:
train_X

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4
65,6.7,3.1,4.4,1.4
11,4.8,3.4,1.6,0.2
42,4.4,3.2,1.3,0.2
...,...,...,...,...
71,6.1,2.8,4.0,1.3
106,4.9,2.5,4.5,1.7
14,5.8,4.0,1.2,0.2
92,5.8,2.6,4.0,1.2


In [231]:
train_y.unique()

array([0, 1, 2])

In [232]:
train_y.value_counts()

1    41
0    40
2    39
Name: Species, dtype: int64

In [233]:
"""def make_input_fn(features,label,training=True,batch_size=32):
    def input_fn():
        dataset=tf.data.Dataset.from_tensor_slices((dict(features),label))
        if training:
            dataset=dataset.shuffle(1000).repeat()
        return dataset.batch(batch_size)
    return input_fn"""

'def make_input_fn(features,label,training=True,batch_size=32):\n    def input_fn():\n        dataset=tf.data.Dataset.from_tensor_slices((dict(features),label))\n        if training:\n            dataset=dataset.shuffle(1000).repeat()\n        return dataset.batch(batch_size)\n    return input_fn'

In [234]:
def input_fn(features,label,training=True,batch_size=32):
    dataset=tf.data.Dataset.from_tensor_slices((dict(features),label))
    if training:
        dataset=dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [235]:
feature_columns=[]
for feature_name in train_X.columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name))

In [236]:
feature_columns

[NumericColumn(key='SepalLengthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLengthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidthCm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## NEEP NEURAL NETWORK CALSSIFIER FOR CLASSFICATION

In [237]:
dnn_classifier=tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[30,10],n_classes=len(Labled_Species))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ALIHAI~1\\AppData\\Local\\Temp\\tmpwoyu11ph', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [238]:
#train_input_fn=make_input_fn(train_X,train_y,training=True)
#dnn_classifier.train(train_input_fn,steps=18000)

In [239]:
dnn_classifier.train(input_fn=lambda:input_fn(train_X,train_y,training=True),steps=18000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\ALIHAI~1\AppData\Local\Temp\tmpwoyu11ph\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1747036, step = 0
INFO:tensorflow:global_step/sec: 1055.51
INFO:tensorflow:loss = 1.0847044, step = 100 (0.094 sec)
INFO:tensorflow:global_step/sec: 2046.28
INFO:tensorflow:loss = 1.0204458, step = 200 (0.050 sec)
INFO:tensorflow:global_step/sec: 2161.32
INFO:tensorflow:loss = 0.977804, step = 300 (0.046 sec)
INFO:tensorflow:global_step/sec: 2228.14
INFO:tensorflow:loss = 0.9371786, step = 400 (0.045 sec)
INFO:tensorflow:global_step/sec: 2278.83
INFO:tensorflow:loss = 0.9354075

INFO:tensorflow:loss = 0.46481222, step = 7800 (0.040 sec)
INFO:tensorflow:global_step/sec: 2387.31
INFO:tensorflow:loss = 0.44929707, step = 7900 (0.042 sec)
INFO:tensorflow:global_step/sec: 2387.32
INFO:tensorflow:loss = 0.41776437, step = 8000 (0.042 sec)
INFO:tensorflow:global_step/sec: 2445.54
INFO:tensorflow:loss = 0.45491004, step = 8100 (0.042 sec)
INFO:tensorflow:global_step/sec: 2278.84
INFO:tensorflow:loss = 0.49165785, step = 8200 (0.044 sec)
INFO:tensorflow:global_step/sec: 2331.8
INFO:tensorflow:loss = 0.50075555, step = 8300 (0.043 sec)
INFO:tensorflow:global_step/sec: 2331.8
INFO:tensorflow:loss = 0.4684682, step = 8400 (0.043 sec)
INFO:tensorflow:global_step/sec: 2228.1
INFO:tensorflow:loss = 0.3681207, step = 8500 (0.045 sec)
INFO:tensorflow:global_step/sec: 1891.86
INFO:tensorflow:loss = 0.48316318, step = 8600 (0.053 sec)
INFO:tensorflow:global_step/sec: 2133.4
INFO:tensorflow:loss = 0.44833285, step = 8700 (0.047 sec)
INFO:tensorflow:global_step/sec: 2228.09
INFO:t

INFO:tensorflow:loss = 0.34082428, step = 16000 (0.044 sec)
INFO:tensorflow:global_step/sec: 2387.37
INFO:tensorflow:loss = 0.31976888, step = 16100 (0.042 sec)
INFO:tensorflow:global_step/sec: 2133.3
INFO:tensorflow:loss = 0.3218733, step = 16200 (0.046 sec)
INFO:tensorflow:global_step/sec: 2331.8
INFO:tensorflow:loss = 0.35307336, step = 16300 (0.043 sec)
INFO:tensorflow:global_step/sec: 1519.21
INFO:tensorflow:loss = 0.34029433, step = 16400 (0.068 sec)
INFO:tensorflow:global_step/sec: 2133.34
INFO:tensorflow:loss = 0.33495975, step = 16500 (0.046 sec)
INFO:tensorflow:global_step/sec: 2228.16
INFO:tensorflow:loss = 0.36003578, step = 16600 (0.045 sec)
INFO:tensorflow:global_step/sec: 2331.84
INFO:tensorflow:loss = 0.32108182, step = 16700 (0.043 sec)
INFO:tensorflow:global_step/sec: 2387.32
INFO:tensorflow:loss = 0.3517716, step = 16800 (0.041 sec)
INFO:tensorflow:global_step/sec: 2387.29
INFO:tensorflow:loss = 0.3511371, step = 16900 (0.043 sec)
INFO:tensorflow:global_step/sec: 233

In [240]:
results=dnn_classifier.evaluate(input_fn=lambda:input_fn(val_X,val_y,training=False))
print(results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-09-05T18:05:55
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ALIHAI~1\AppData\Local\Temp\tmpwoyu11ph\model.ckpt-18000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18553s
INFO:tensorflow:Finished evaluation at 2023-09-05-18:05:55
INFO:tensorflow:Saving dict for global step 18000: accuracy = 0.96666664, average_loss = 0.32600984, global_step = 18000, loss = 0.32600984
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18000: C:\Users\ALIHAI~1\AppData\Local\Temp\tmpwoyu11ph\model.ckpt-18000
{'accuracy': 0.96666664, 'average_loss': 0.32600984, 'loss': 0.32600984, 'global_step': 18000}


## PREDICTION

In [241]:
def input_fn(features,batch_size=32):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [242]:
new_data = {
    'SepalLengthCm': [5.1, 6.2],
    'SepalWidthCm': [3.5, 2.9],
    'PetalLengthCm': [1.4, 4.3],
    'PetalWidthCm': [0.2, 1.3],
}
new_data=pd.DataFrame(new_data)
new_data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,6.2,2.9,4.3,1.3


In [243]:
predictions = dnn_classifier.predict(input_fn=lambda: input_fn(new_data))
predictions

<generator object Estimator.predict at 0x000002873853B430>

In [244]:
for p in predictions:
    class_ids=p['class_ids'][0]
    probability=p['probabilities'][class_ids]
    decoded_labels = label_encoder.inverse_transform([class_ids])
    print('Class : ',decoded_labels,' with probability of',probability*100,'%')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ALIHAI~1\AppData\Local\Temp\tmpwoyu11ph\model.ckpt-18000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Class :  ['Iris-setosa']  with probability of 95.64289450645447 %
Class :  ['Iris-versicolor']  with probability of 64.22238349914551 %
